<a href="https://colab.research.google.com/github/kerenalli/DATA_MINING/blob/main/20210120_Eye_Blink_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Necessary imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
from google.colab.patches import cv2_imshow

In [37]:
# Downloading and unzipping facial landmark predictor
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 "shape_predictor_68_face_landmarks.dat.bz2"

--2021-01-20 11:46:34--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.1’

shape_predictor_68_ 100%[===================>]  61.07M  18.4MB/s    in 4.0s    

2021-01-20 11:46:38 (15.4 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.1’ saved [64040097/64040097]

bunzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


In [38]:
 # Initializing the face detector and facial landmark predictor 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [39]:
# Creating a VidoCapture and VideoWriter object
cap=cv2.VideoCapture("/content/drive/MyDrive/Classification/Videos/modi.mp4")
fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
out = cv2.VideoWriter('output.mp4', fourcc, 29, (540,620))
font=cv2.FONT_HERSHEY_SIMPLEX

In [40]:
# Defining the mid-point
def midpoint(p1 ,p2):
  return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

In [42]:
# Defining the eye aspect ratio
def get_EAR(eye_points, facial_landmarks):
# Defining the left point of the eye   
    left_point = [facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y]
# Defining the right point of the eye   
    right_point = [facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y]
# Defining the top mid-point of the eye    
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
# Defining the bottom mid-point of the eye   
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))
# Drawing horizontal and vertical line       
    hor_line = cv2.line(frame, (left_point[0], left_point[1]), (right_point[0], right_point[1]), (255, 0, 0), 3)
    ver_line = cv2.line(frame, (center_top[0], center_top[1]),(center_bottom[0], center_bottom[1]), (255, 0, 0), 3)
 # Calculating length of the horizontal and vertical line    
    hor_line_lenght = euclidean_distance(left_point[0], left_point[1], right_point[0], right_point[1])
    ver_line_lenght = euclidean_distance(center_top[0], center_top[1], center_bottom[0], center_bottom[1])
 # Calculating eye aspect ratio     
    EAR = ver_line_lenght / hor_line_lenght
    return EAR

In [44]:
def mouth_aspect_ratio(mouth):
	# compute the euclidean distances between the two sets of
	# vertical mouth landmarks (x, y)-coordinates
	A = dist.euclidean(mouth[2], mouth[9]) # 51, 59
	B = dist.euclidean(mouth[4], mouth[7]) # 53, 57

	# compute the euclidean distance between the horizontal
	# mouth landmark (x, y)-coordinates
	C = dist.euclidean(mouth[0], mouth[6]) # 49, 55

	# compute the mouth aspect ratio
	mar = (A + B) / (2.0 * C)

	# return the mouth aspect ratio
	return mar

In [35]:
# Creating a list eye_blink_signal
eye_blink_signal=[]
# Creating an object blink_ counter
blink_counter = 0
previous_ratio = 100
# Creating a while loop
while True:
  ret, frame = cap.read() 
  if ret == False:
    break
# Converting a color frame into a grayscale frame  
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Creating an object in which we will sore detected faces  
  faces = detector(gray)
  for face in faces:
    x, y = face.left(), face.top()
    x1, y1 = face.right(), face.bottom()
# Creating an object in which we will sore detected facial landmarks
    landmarks = predictor(gray, face)
# Calculating left eye aspect ratio    
    left_eye_ratio = get_EAR([36, 37, 38, 39, 40, 41], landmarks)
# Calculating right eye aspect ratio  
    right_eye_ratio = get_EAR([42, 43, 44, 45, 46, 47], landmarks)
# Calculating aspect ratio for both eyes  
    blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
# Rounding blinking_ratio on two decimal places   
    blinking_ratio_1 = blinking_ratio * 100
    blinking_ratio_2 = np.round(blinking_ratio_1)
    blinking_ratio_rounded = blinking_ratio_2 / 100
# Appending blinking ratio to a list eye_blink_signal
    eye_blink_signal.append(blinking_ratio)
    if blinking_ratio < 0.20:
      if previous_ratio > 0.20:
        blink_counter = blink_counter + 1
 # Displaying blink counter and blinking ratio in our output video      
    
    previous_ratio = blinking_ratio
  
  cv2.putText(frame, str(blink_counter), (30, 50), font, 2, (0, 0, 255),5)
  cv2.putText(frame, str(blinking_ratio_rounded), (900, 50), font, 2, (0, 0, 255),5)
  out.write(frame)
out.release()

In [43]:
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/content/drive/MyDrive/Classification/Videos/modi.mp4')
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=640 controls/>""")

Requirement already up-to-date: kora in /usr/local/lib/python3.6/dist-packages (0.9.15)
